In [1]:
# This is the training script for fine-tuning bert on 
# unaltered GPT data and manually labelled data
# For better performance/generalization, look for augmented dataset
# Read README.md for comments and details.

In [2]:
# all classes

classes = ["banking","valuation","household","real estate","corporate","external","sovereign","technology", "climate", "energy", "health", "eu"]


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,top_k_accuracy_score
import math
import pickle
from datasets import Dataset

In [4]:
# load bert-based and finbert
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(classes))
finbert = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert', use_fast =True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
# weights transfer for encoder layers only 
finbert_weights = finbert.state_dict()
model_weights = model.state_dict()
del finbert_weights["bert.pooler.dense.weight"]
del finbert_weights["bert.pooler.dense.bias"]
del finbert_weights["classifier.weight"]
del finbert_weights["classifier.bias"]
finbert_weights["bert.pooler.dense.weight"] = model_weights["bert.pooler.dense.weight"]
finbert_weights["bert.pooler.dense.bias"] = model_weights["bert.pooler.dense.bias"]
finbert_weights["classifier.weight"] = model_weights["classifier.weight"]
finbert_weights["classifier.bias"] = model_weights["classifier.bias"]

model.load_state_dict(finbert_weights)


<All keys matched successfully>

In [6]:
# flatten to one list for all 3

# manual labelled
with open('train_data.pickle', 'rb') as file:
    train = pickle.load(file)

# gpt labelled p1
with open('gpt.pickle', 'rb') as file:
    gpt = pickle.load(file)

# gpt labelled p2
with open('gpt_p2.pickle', 'rb') as file:
    gpt2 = pickle.load(file)
    
gpt = [item for sublist in gpt for item in sublist]
gpt2 = [item for sublist in gpt2 for item in sublist]

mixed = gpt + gpt2

In [7]:
print(len(mixed))

2458


In [8]:
# change to sampling methods instead of argmax if treat GPT-3 data as probabilities.
sample = 1

text_max = [item["text"] for i in range(sample) for item in train]
label_max = [np.argmax(item["dist"]) for i in range(sample) for item in train]

text_max_mixed = [item["text"] for i in range(sample) for item in mixed]
label_max_mixed = [np.argmax(item["dist"]) for i in range(sample) for item in mixed]

comb_text = text_max_mixed
comb_label = label_max_mixed

In [9]:
assert len(comb_text) == len(comb_label)

In [10]:
# random shuffle
import random
temp = list(zip(comb_text, comb_label))
random.shuffle(temp)
comb_text, comb_label = zip(*temp)

In [11]:
print(len(comb_text))

2458


In [12]:
train_text, test_text = comb_text, text_max
train_label, test_label = comb_label, label_max

In [13]:
train_dataset = Dataset.from_dict({"text":train_text, "label":train_label})
test_dataset = Dataset.from_dict({"text":test_text, "label":test_label})

In [14]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

100%|██████████| 1/1 [00:00<00:00, 66.65ba/s]


In [15]:
train_dataset["input_ids"]

tensor([[ 101, 2023, 2095,  ...,    0,    0,    0],
        [ 101, 1999, 2240,  ...,    0,    0,    0],
        [ 101, 2023, 2001,  ...,    0,    0,    0],
        ...,
        [ 101, 9308, 1010,  ...,    0,    0,    0],
        [ 101, 9308, 1010,  ...,    0,    0,    0],
        [ 101, 2021, 2062,  ...,    0,    0,    0]])

In [16]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    top3 = top_k_accuracy_score(labels, pred.predictions,k=3)
    top2 = top_k_accuracy_score(labels, pred.predictions,k=2)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'top3': top3,
         'top2': top2
    }

training_args = TrainingArguments(
    
    output_dir='./results',
    learning_rate=2e-5,
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    metric_for_best_model="accuracy",
    evaluation_strategy='epoch',
    save_strategy = "epoch",
    logging_dir='./logs',
    save_total_limit = 1, # Only last 5 models are saved. Older ones are deleted.
    load_best_model_at_end=True,
)
    
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [17]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 2458
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3080
  5%|▍         | 153/3080 [00:16<05:41,  8.58it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 261
  Batch size = 64
C:\Users\felix\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

{'eval_loss': 2.509948968887329, 'eval_accuracy': 0.13793103448275862, 'eval_f1': 0.046496732026143794, 'eval_precision': 0.0320860240449657, 'eval_recall': 0.10346320346320344, 'eval_top3': 0.3448275862068966, 'eval_top2': 0.21839080459770116, 'eval_runtime': 0.3794, 'eval_samples_per_second': 687.981, 'eval_steps_per_second': 13.18, 'epoch': 1.0}


Model weights saved in ./results\checkpoint-154\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-765] due to args.save_total_limit
 10%|▉         | 307/3080 [00:37<05:34,  8.30it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 261
  Batch size = 64
C:\Users\felix\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 10%|█         | 308/3080 [00:38<05:33,  8.30it/s]Saving model checkpoint to ./results\checkpoint-308
Configuration saved in ./results\checkpoint-308\config.json


{'eval_loss': 2.377951145172119, 'eval_accuracy': 0.2796934865900383, 'eval_f1': 0.14517426296872502, 'eval_precision': 0.11616984437030764, 'eval_recall': 0.24255256196045671, 'eval_top3': 0.4674329501915709, 'eval_top2': 0.3793103448275862, 'eval_runtime': 0.3759, 'eval_samples_per_second': 694.411, 'eval_steps_per_second': 13.303, 'epoch': 2.0}


Model weights saved in ./results\checkpoint-308\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-3060] due to args.save_total_limit
 15%|█▌        | 462/3080 [01:02<05:49,  7.48it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 261
  Batch size = 64
C:\Users\felix\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 15%|█▌        | 462/3080 [01:02<05:49,  7.48it/s]Saving model checkpoint to ./results\checkpoint-462
Configuration saved in ./results\checkpoint-462\config.json


...

100%|██████████| 3080/3080 [07:43<00:00, 10.62it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 261
  Batch size = 64
C:\Users\felix\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

100%|██████████| 3080/3080 [07:43<00:00, 10.62it/s]Saving model checkpoint to ./results\checkpoint-3080
Configuration saved in ./results\checkpoint-3080\config.json


{'eval_loss': 2.6133997440338135, 'eval_accuracy': 0.5632183908045977, 'eval_f1': 0.5221213693587845, 'eval_precision': 0.5603289932573821, 'eval_recall': 0.5580912508544087, 'eval_top3': 0.8275862068965517, 'eval_top2': 0.7662835249042146, 'eval_runtime': 0.3341, 'eval_samples_per_second': 781.26, 'eval_steps_per_second': 14.967, 'epoch': 20.0}


Model weights saved in ./results\checkpoint-3080\pytorch_model.bin
Deleting older checkpoint [results\checkpoint-2926] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-2618 (score: 0.5747126436781609).
100%|██████████| 3080/3080 [07:46<00:00,  6.61it/s]

{'train_runtime': 466.0765, 'train_samples_per_second': 105.476, 'train_steps_per_second': 6.608, 'train_loss': 0.48012792072319366, 'epoch': 20.0}


TrainOutput(global_step=3080, training_loss=0.48012792072319366, metrics={'train_runtime': 466.0765, 'train_samples_per_second': 105.476, 'train_steps_per_second': 6.608, 'train_loss': 0.48012792072319366, 'epoch': 20.0})

In [18]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 261
  Batch size = 64
 80%|████████  | 4/5 [00:00<00:00, 11.60it/s]C:\Users\felix\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 5/5 [00:00<00:00, 14.32it/s]


{'eval_loss': 2.512448310852051,
 'eval_accuracy': 0.5747126436781609,
 'eval_f1': 0.5442073581817862,
 'eval_precision': 0.5868235712180875,
 'eval_recall': 0.5721994379889117,
 'eval_top3': 0.8237547892720306,
 'eval_top2': 0.7701149425287356,
 'eval_runtime': 0.4489,
 'eval_samples_per_second': 581.427,
 'eval_steps_per_second': 11.138,
 'epoch': 20.0}